In [54]:
# Dependencies
import os
import matplotlib.cm as cm
import pandas as pd
import shutil
pd.options.mode.chained_assignment = None  # default='warn'


## Firm Data Cleaning by Jane Wallace
* #### Get, loop thru all files to find the correct excel sheets (2012-2016)
* #### Format, clean, and filter data
* #### If error occurs, write to text file and read to display
* #### Save result to output path

In [55]:
# Set directory file paths
filepath = os.path.join(".", "Resources/Business_States_2012-2016")
output_path = os.path.join(".", "Output", "data_clean_error.txt")
output_co_ttl = os.path.join(".", "Output", "state_co_ttl_2012-16.csv")
output_co_grp = os.path.join(".", "Output", "state_co%_2012-16.csv")
output_co_ind = os.path.join(".", "Output", "state_co_ind_ttl_2012-16.csv")
output_co_ind_grp = os.path.join(".", "Output", "state_co_ind_grp_2012-16.csv")

# Read all files from directory filepath
xList = [x for x in os.listdir(filepath)]


In [56]:
# Open text file in APPEND mode (a, r - read, w -write)
with open(output_path, "a") as xfile:
    i = 1
    
    # Loop thru the file list and write info to opened text file if criteria are met
    for x in xList:                
        # If error, record exception
        try:
            # Split the extension from the path and normalise it to lowercase.
            ext = os.path.splitext(x)[-1].lower()
            excel = os.path.splitext(x)[0].lower()

            # If it's excel and contain string below, then get year, read file,  
            # format data by year on # of Companys column, selected certain columns, and merge data
            if "state_naicssector" in x:
                if ext == ".xls" or ext == ".xlsx": 
                    yr = excel.split("_")[2]

                    business_data = pd.read_excel(f'{filepath}/{x}' , skiprows=5)

                    business_data = business_data.drop(business_data.index[[0,1]])
                    splitted_col = business_data["ENTERPRISE EMPLOYMENT SIZE"].str.split(":", n = 1, expand = True) 

                    # Split enterprise employment size to # & size cols
                    business_data["Num"]= splitted_col[0].str.strip()   
                    business_data["Co_Size"]= splitted_col[1].str.strip()
                  
                    # Select data columns
                    state_co = business_data[["STATE DESCRIPTION", "NUMBER OF FIRMS", "NAICS DESCRIPTION", "Num", "Co_Size"]]
                    state_co.columns = ["State", yr + "_#Companies", "NAICS_Desc", "Num(#)", "Company_Size"]
                    state_co[yr + "_#Companies"] = state_co[yr + "_#Companies"].astype(int)

                    # Filter data by state total # of companies
                    state_total = state_co.loc[(state_co["NAICS_Desc"] == "Total") & (state_co["Company_Size"] == "Total")]
                    state_co_ttl = state_total[["State", yr + "_#Companies"]]
                    
                    # -------------------------------------------------------------------------------------------
                    # Filter data by state % of small & large company group 
                    state_detail = state_co.loc[(state_co["NAICS_Desc"] == "Total") & (~state_co["Company_Size"].str.contains('<')) & (state_co["Company_Size"] != "Total")]
                    company_total = state_detail.groupby("State")
                    company_total = company_total[yr + "_#Companies"].sum()

                    state_detail[yr + "_Co_Size%"] =  [round((state[yr + "_#Companies"]/company_total[state["State"]]) * 100,1)  for i, state in state_detail.iterrows()]
                    state_co_grp = state_detail[["State", "Company_Size", yr + "_#Companies", yr + "_Co_Size%"]]
    
                    # -------------------------------------------------------------------------------------------
                    # Filter data by state total # of companies in 20 different industies
                    co_ttl_industry = state_co.loc[(state_co["NAICS_Desc"] != "Total") & (state_co["Company_Size"] == "Total")]
                    co_ttl_industry = co_ttl_industry.reset_index(drop=True)
                    co_ttl_industry = co_ttl_industry[["State", "NAICS_Desc", yr + "_#Companies"]].rename(columns={"NAICS_Desc": "Industry"})
                    
                    # -------------------------------------------------------------------------------------------
                    # Filter data by state total # of companies in 20 different industies
                    co_ind_grp = state_co.loc[(state_co["NAICS_Desc"] != "Total") & (~state_co["Company_Size"].str.contains('<')) & (state_co["Company_Size"] != "Total")]
                    co_ind_grp = co_ind_grp.reset_index(drop=True)
                    co_ind_grp = co_ind_grp[["State", "NAICS_Desc", "Company_Size", yr + "_#Companies"]].rename(columns={"NAICS_Desc": "Industry"})
                    
                    # -------------------------------------------------------------------------------------------
                    # Merge data by state
                    if i == 1:
                        co_ttl_df = pd.DataFrame(state_co_ttl)
                        co_grp_pct_df =  pd.DataFrame(state_co_grp)
                        ind_ttl_df =  pd.DataFrame(co_ttl_industry)
                        ind_grp_df =  pd.DataFrame(co_ind_grp)
                    else:
                        co_ttl_df = pd.merge(co_ttl_df, state_co_ttl, how='inner', on='State')
                        co_grp_pct_df = pd.merge(co_grp_pct_df , state_co_grp, on=['State','Company_Size'])
                        ind_ttl_df = pd.merge(ind_ttl_df, co_ttl_industry, on=['State','Industry'])
                        ind_grp_df = pd.merge(ind_grp_df, co_ind_grp, on=['State','Industry', 'Company_Size'])
                    
                    i += 1    # Increment by 1 as excel sheet is cleaned and formatted
        except Exception as e:
            # Write error to text file if occurs
            xfile.write(f'****************************************************************************\n')
            xfile.write(f'### {x} ###\n')  # Write file name to text file            
            xfile.write("Exception occurred: {}".format(e) + "\n")  # Write error to text file


# Dataset result - count columns' rows
co_ttl_df.count()


State              51
2012_#Companies    51
2013_#Companies    51
2014_#Companies    51
2015_#Companies    51
2016_#Companies    51
dtype: int64

In [57]:
# Dataset result - count columns' rows
co_grp_pct_df.count()

State              306
Company_Size       306
2012_#Companies    306
2012_Co_Size%      306
2013_#Companies    306
2013_Co_Size%      306
2014_#Companies    306
2014_Co_Size%      306
2015_#Companies    306
2015_Co_Size%      306
2016_#Companies    306
2016_Co_Size%      306
dtype: int64

In [58]:
# Dataset result - count columns' rows
ind_ttl_df.count()

State              1019
Industry           1019
2012_#Companies    1019
2013_#Companies    1019
2014_#Companies    1019
2015_#Companies    1019
2016_#Companies    1019
dtype: int64

In [59]:
# Dataset result - count columns' rows
ind_grp_df.count()

State              5866
Industry           5866
Company_Size       5866
2012_#Companies    5866
2013_#Companies    5866
2014_#Companies    5866
2015_#Companies    5866
2016_#Companies    5866
dtype: int64

In [60]:
# Open output file & print result if error occurs
if os.path.exists(output_path):
    print(open(output_path).read())

****************************************************************************
### state_naicssector_2013.xlsx ###
Exception occurred: 'NAICS_Desc'
****************************************************************************
### state_naicssector_2014.xlsx ###
Exception occurred: 'NAICS_Desc'
****************************************************************************
### state_naicssector_2015.xlsx ###
Exception occurred: 'NAICS_Desc'
****************************************************************************
### state_naicssector_2016.xlsx ###
Exception occurred: 'NAICS_Desc'
****************************************************************************
### state_naicssector_2012.xlsx ###
Exception occurred: "['NAICS_Desc' 'Company_Size'] not in index"
****************************************************************************
### state_naicssector_2013.xlsx ###
Exception occurred: "['NAICS_Desc' 'Company_Size'] not in index"
************************************************************

In [61]:
# Save cleaned data to csv & display
# Total Companies in each state
co_ttl_df.to_csv(output_co_ttl)
co_ttl_df.head()

,State,2012_#Companies,2013_#Companies,2014_#Companies,2015_#Companies,2016_#Companies
0,Alabama,73661,73371,73154,73409,73976
1,Alaska,16656,16733,16841,16952,17059
2,Arizona,101205,102313,103312,105004,106963
3,Arkansas,50412,50318,50151,50451,50511
4,California,701899,711086,724483,740303,751982


In [62]:
# Save cleaned data to csv & display
# Company size total & percentage by state 
co_grp_pct_df["Company_Size"] = [(f'\{percent["Company_Size"]}') for i, percent in co_grp_percent.iterrows()]
co_grp_pct_df.to_csv(output_co_grp)
co_grp_pct_df.head()

,State,Company_Size,2012_#Companies,2012_Co_Size%,2013_#Companies,2013_Co_Size%,2014_#Companies,2014_Co_Size%,2015_#Companies,2015_Co_Size%,2016_#Companies,2016_Co_Size%
0,Alabama,\\0-4,39993,54.3,39890,54.4,39562,54.1,39724,54.1,39880,53.9
1,Alabama,\\5-9,13803,18.7,13622,18.6,13603,18.6,13646,18.6,13721,18.5
2,Alabama,\\10-19,8277,11.2,8199,11.2,8215,11.2,8205,11.2,8281,11.2
3,Alabama,\\20-99,7280,9.9,7258,9.9,7360,10.1,7358,10.0,7539,10.2
4,Alabama,\\100-499,1926,2.6,1977,2.7,1974,2.7,1981,2.7,2017,2.7


In [63]:
# Save cleaned data to csv & display
# Total Companies in each state
ind_ttl_df.to_csv(output_co_ind)
ind_ttl_df.head()

,State,Industry,2012_#Companies,2013_#Companies,2014_#Companies,2015_#Companies,2016_#Companies
0,Alabama,"Agriculture, Forestry, Fishing and Hunting",770,780,782,775,796
1,Alabama,"Mining, Quarrying, and Oil and Gas Extraction",204,189,180,171,160
2,Alabama,Utilities,111,111,111,110,110
3,Alabama,Construction,7347,7221,7146,7184,7292
4,Alabama,Manufacturing,3865,3781,3761,3749,3703


In [64]:
# Save cleaned data to csv & display
# Total Companies in each state
ind_grp_df.to_csv(output_co_ind_grp)
ind_grp_df.head()

,State,Industry,Company_Size,2012_#Companies,2013_#Companies,2014_#Companies,2015_#Companies,2016_#Companies
0,Alabama,"Agriculture, Forestry, Fishing and Hunting",0-4,385,379,386,386,415
1,Alabama,"Agriculture, Forestry, Fishing and Hunting",5-9,222,233,234,227,218
2,Alabama,"Agriculture, Forestry, Fishing and Hunting",10-19,98,103,98,91,95
3,Alabama,"Agriculture, Forestry, Fishing and Hunting",20-99,47,47,46,54,49
4,Alabama,"Agriculture, Forestry, Fishing and Hunting",100-499,6,6,8,7,8
